<a href="https://colab.research.google.com/github/jugernaut/ProgramacionEnParalelo/blob/desarrollo/OpenMP/03_Aproximacion_Pi_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Aproximación Pi (OpemMP)</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M.en.C. Miguel Angel Pérez León</i></h5>
    <h5 align="center"><i>Ayudante: Jesús Iván Coss Calderón</i></h5>
    <h5 align="center"><i>Ayudante: Mario Arturo Nieto Butron</i></h5>
  <h5 align="center"><i>Materia: Seminario de programación en paralelo</i></h5>
  </font>

# Introducción

Un valor muy recurrente en muchas áreas de la ciencia es el valor de $\pi=3.1415......$, por lo que mostrar una forma de aproximar este valor por métodos numéricos (y sobretodo en paralelo) es un ejercicio tanto interesante como didáctico.

Desde comienzos de la humanidad han surgido un sin número de algoritmos para aproximar este valor, entre los más interesantes destacan, el método egipcio, babilónico, chino, mesopotámico, Gauss, etc.

En épocas más recientes han surgido métodos de aproximación numérica más sofisticados con lo son los métodos de Monte Carlo, sin embargo este tipo de métodos serán tratados más adelante.

Para esta presentación se empleara otra de las definiciones en el campo de las ciencias e ingeniería que tiene un amplio espectro de aplicación, este es el concepto de la **Integral Definida**.

$$ \int_{a}^{b}f\left(x\right)=F\left(b\right)-F\left(a\right)$$

Al igual que la derivada de una función, la integral tiene múltiples aplicaciones, está presente en una infinidad de áreas y sus aplicaciones van desde probabilidades, pasando por centros de masa y hasta volúmenes, por mencionar algunas.

## Planteamiento

Supongamos que se quiere dar una aproximación numérica del valor de $\pi$, por lo tanto es necesario plantear que se necesita para dar tal aproximación. 

Por simplicidad supongamos que se tiene una circunferencia de radio 1 $\left(r=1\right)$, es decir 

$$\sqrt{x^{2}+y^{2}}=1 \tag{1}$$

Ahora si $r=1$ entonces el área de esta circunferencia se calcula mediante la relación 
$$A=\pi r^{2}\Rightarrow\pi=\frac{A}{r^{2}}\Rightarrow\pi=\frac{A}{\left(1\right)\left(1\right)}\Rightarrow\pi=A \tag{2}$$

De $\left(2\right)$ se puede concluir que al dar aproximación del área de esta circunferencia, estaremos dando una aproximación del valor del número $\pi$.

De $\left(1\right)$ podemos quedarnos con una semicircunferencia de radio 1, es decir 

$$\sqrt{1-x^{2}} \tag{3}$$

Ahora, de nuevo por simplicidad calculemos el área bajo la curva de $\left(3\right)$ desde 0 hasta 1 

$$\int_{0}^{1}\sqrt{1-x^{2}}dx \tag{4}$$

Esta área seria $\frac{1}{4}$ del área total de la circunferencia de radio 1, por lo que si queremos dar una aproximación de $\pi$ es equivalente a dar una aproximación de la integral definida en $\left(4\right)$ y finalmente multiplicar por 4 para encontrar el área total.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/OpenMP/circunferencia.png?raw=1" width="400"> 
</center>

# Integral Numérica

Existen múltiples formas de emplear métodos numéricos para dar una aproximación de la integral definida, por ejemplo punto medio, trapecio, Simpson, etc.

Para este ejemplo vamos a utilizar las sumas de **Reimann** como primer método para aproximar el valor de $\pi$.

## Repaso de cálculo

La Integral puede ser definida de muchas formas, sin embargo la forma mas sencilla de verla es mediante las sumas de Riemann, lo cual consiste en trazar un número finito de rectángulos dentro de un área irregular, calcular el área de cada uno de ellos y sumarlos.

$$\int_{a}^{b}f\left(x\right)dx=\underset{\triangle x\rightarrow0}{lim}\sum_{i=1}^{n}f\left(x_{i}\right)\triangle x_{i}$$

Por lo que para dar la aproximación de la integral numérica de cualquier función es suficiente con calcular las áreas de estos rectángulos bajo la curva y finalmente sumar dichas áreas.

Vale la pena notar que **el calculo de cada una de estas áreas no depende de ningún calculo previo**, es decir que esta aproximación puede ser calculada en paralelo.

Sin embargo al realizar esta aproximación de esta manera se tiene un gran problema, el error de aproximación ya que lo preciso de esta aproximación depende de $\triangle x$ (delta equis), existen mejores formas de realizar esta aproximación, pero para comenzar utilizaremos esta. 

## Algoritmo básico

A continuación se muestra el pseudocódigo del algoritmo básico para la aproximación del valor del número $\pi$.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/OpenMP/aproxpi.png?raw=1" width="600"> 
</center>

In [6]:
# import de biblioteca calculos numericos
import math

# aproximacion de Pi mediante sumas de Reimann
def aproxPI(n=1000):
    suma=0
    # funcion a integrar
    f = lambda x: math.sqrt(1-x**2)
    delta = 1/n
    for i in range(1,n+1):
        x = delta*i
        suma += delta*f(x)
    aproximacion = suma*4
    return aproximacion

# pruebas del algoritmo inicial
print(aproxPI())


3.141572616401957


# Aproximación de $\pi$

Podemos concluir que una aproximación elemental para el valor de $\pi$ está dada por el algoritmo anterior.

Sin embargo hay muchas cosas que podemos notar de dicho algoritmo, principalmente que este, es un algoritmo secuencial. ¿Sera posible encontrar una forma en paralelo de dicho algoritmo?.

Otra de las cosas que podemos notar es el numero de operaciones (complejidad computacional) que se están realizando. ¿Se podrá reducir este número de operaciones?.

El error de aproximación es demasiado alto. ¿cómo podemos disminuir este error de aproximación?.

## Análisis

La idea principal para disminuir el número de operaciones y por lo tanto reducir el tiempo de ejecución de nuestro algoritmo se basa en uno de los axiomas de las matemáticas.

$$ab+ac=a\left(b+c\right)$$

Pasamos de realizar **3 operaciones** del lado izquierdo de la igualdad (2 multiplicaciones y una suma) **a 2 operaciones** del lado derecho de la igualdad (una suma y una multiplicación).

Después para disminuir el error de aproximación podemos emplear algoritmos para aproximar integrales un poco más precisos, como seria el método de los **trapecios o punto medio**.

Finalmente para aumentar la velocidad con la que se ejecuta este algoritmo podemos emplear el concepto de **paralelización incremental**, lo que significa que a partir del algoritmo secuencial podemos convertirlo en un algoritmo en paralelo gradualmente.

## Condición de Carrera

Supongamos que tenemos el código secuencial del algoritmo para la aproximación del valor de $\pi$. Para convertirlo de manera gradual en un algoritmo en paralelo empleando OpenMP, es necesario agregar en la linea 3, la directiva de compilador

$\color{red}{\#pragma }$ $\color{red}{omp}$ $\color{red}{parallel}$ $\color{red}{ for}$

Esto con la idea de que **el ciclo for sea ejecutado por múltiples hilos** en paralelo, sin embargo esto nos conduce a un problema frecuente al programar en paralelo conocido como **condición de carrera**.

Esto significa que muchos hilos están compitiendo por acceder (y potencialmente) actualizar una variable compartida, lo cual puede llevar a cálculos erróneos.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/AproximacionPi/condicion.png?raw=1" width="600"> 
</center>

OpenMP ofrece múltiples mecanismos para evitar que problemas como la condición de carrera afecte en el resultado de los cálculos, uno de estos mecanismos consiste en definir secciones criticas con la directiva.

$\color{red}{\#pragma }$ $\color{red}{omp}$ $\color{red}{critical}$

Esta directiva le indica al compilador que **solo un hilo a la vez puede acceder a esta sección**, de tal manera que podemos indicar que el valor del área que se esta calculando solo sea actualizada por un hilo a la vez.

Después de realizar estos cambios en el código, este se vería así.







# Algortimo en paralelo

A continuación se muestra el pseudocódigo del algoritmo en paralelo para la aproximación del valor del número $\pi$.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/AproximacionPi/alg_par.PNG?raw=1" width="600"> 
</center>

Sin embargo esta nueva versión aun cuenta con algunos problemas, aunque ya se ejecuta en paralelo los problemas son los siguientes.

Aun se tiene el problema del error de aproximación, es decir que es necesario tomar una $\Delta x$ muy pequeña para tener una buena precisión en el calculo ó podríamos emplear la estrategia de los trapecios o punto medio.

Y lo más importante, es que aunque ya evitamos la condición de carrera dado que la sección critica establece que solo un hilo puede actualizar la variable sum, entonces este algoritmo es equivalente a realizar el calculo de las áreas una por una.



In [55]:
codigo = """
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>
#include <math.h>

//gcc -o piparalelomitad -fopenmp piparalelomitad.c -lm
 
int main(){
	int n;
	double PI, delta, sum, error, pi, INICIO, FIN, tiempof;
	printf("introduce la precision del calculo (n > 0): ");
	scanf("%d", &n);
	pi = 0.0;	
	delta = 1.0 / (double) n;
	sum = 0.0;
	#pragma omp parallel for
		for (int i = 1; i <= n; i++) {
		double x = delta * i;
    #pragma omp critical
		sum += delta * (sqrt(1.0 - x*x)); //desperdicio de recursos OJO
	}
	pi = sum * 4;
  printf("El valor aproximado de pi es: %f\\n", pi); 
}
"""
# se crea el archivo con permisos para escribir mediante python
archivo_texto = open("aproxpi.c", "w")
# se escribe el programa en el archivo 
archivo_texto.write(codigo)
# se cierra el buffer de escritura
archivo_texto.close()

In [56]:
%env OMP_NUM_THREADS=3
!gcc -o aprox -fopenmp aproxpi.c -lm

env: OMP_NUM_THREADS=3


In [57]:
!./aprox

introduce la precision del calculo (n > 0): 100
El valor aproximado de pi es: 3.120417


## Reduction

Es por esta razón que existen directivas de compilador como,

$\color{red}{\#pragma }$ $\color{red}{omp}$ $\color{red}{parallel}$ $\color{red}{ for}$ $\color{red}{ reduction(+:sum)}$

Cuya semántica es, ''ejecuta este ciclo for en paralelo y al final del mismo, en la variable *sum* almacena la suma los resultados parciales de haber calculado las respectivas áreas''.

De tal forma que podemos pensar que por cada hilo que ejecuta una iteración del ciclo *for*, se tiene una variable temporal, digamos area_parcial en la cual se almacena el calculo de una de estas áreas.

Al termino el ciclo la variable *sum*, cuenta con la suma de todas las áreas parciales que se calcularon previamente y por lo tanto el área total bajo la curva.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/AproximacionPi/reduction.png?raw=1" width="600"> 
</center>

De esta forma el calculo de todas las áreas se lleva a cabo en paralelo y no es necesario esperar a que se realice hilo por hilo.

Finalmente solo resta utilizar la aproximación del área bajo la curva empleando la regla del trapecio o punto medio, con $n$ numero de intervalos.

$$h=\frac{b-a}{n}=\frac{1-0}{n}=\frac{1}{n}$$

$$\begin{array}{ccc} \int_{a}^{b}f\left(x\right)dx & = & h*f\left(\frac{x_{1}-x_{0}}{2}\right)+\cdots+h*f\left(\frac{x_{n}-x_{n-1}}{2}\right)\\
 & = & h\left(f\left(\frac{x_{1}-x_{0}}{2}\right)+\cdots+f\left(\frac{x_{n}-x_{n-1}}{2}\right)\right)
\end{array}$$

Así que tomando en cuenta todas las mejoras antes mencionadas, el algoritmo para aproximar el valor de $\pi$ seria el siguiente.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/AproximacionPi/alg_par_2.PNG?raw=1" width="600"> 
</center>

## Complejidad

Como ya sabemos este algoritmo a pesar de realizar varias operaciones dentro del ciclo *for*, este número de operaciones es constante y el numero total de operaciones en realidad depende del número de precisión o de intervalos $n$, por lo que podemos decir que este algoritmo pertenece a $O\left(n\right)$

Pero es una buena práctica medir el tiempo de las diferentes versiones del algoritmo e incluso en diferentes equipos de cómputo, por lo que necesitamos emplear una función muy útil dentro de **OpenMP omp_get_wtime()**;

Podemos emplear esta función para calcular el tiempo total que le tomo a nuestro algoritmo realizar el calculo, además si tenemos un referente del valor real de $\pi$ podemos dar una estimación del error cometido por nuestro calculo.

## Versión Final

Pseudocódigo de la versión final del algoritmo en paralelo para la aproximación del valor del número $\pi$.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/AproximacionPi/alg_final.PNG?raw=1" width="600"> 
</center>


# Hilos en alto nivel

Actualmente la mayoría de los lenguajes de alto nivel (como Java) permiten hacer uso de hilos de ejecución (procesos ligeros). Normalmente es a través de una biblioteca, modulo, o paquete que se permite hacer uso de hilos, depende del lenguaje.

Es decir que que ya no hace falta preocuparse por como esta hecha la implementación del manejo de hilos, no es necesario tener conocimientos profundos de como funciona el sistema operativo y mucho menos el ó los microprocesadores, para poder hacer uso de esta poderosa herramienta.

Dado que Java es un lenguaje completamente orientado a objetos, la forma en la cual se puede hacer uso de hilos de ejecución es mediante la clase Thread o la interfaz Runable.


## La interfaz Runnable

Las interfaces son una de las características mas importantes y potentes de java (no solo las graficas), mediante una interfaz de java se puede definir un comportamiento y en el caso de los hilos, podemos definir que una clase debe comportarse como un hilo de ejecución. 

Una vez que la clase en cuestión implementa la interfaz Runnable (implements Runnable), es decir que define el único método de esta interfaz, podemos decir que nuestra clase se puede comportar como un hilo de ejecución, por lo tanto puede ser ejecutado mediante el método run().

Así de simple podemos definir que una determinada clase se comporte como un hilo de ejecución, Considerando el concepto de hilo, el método run() debe ser algo no muy complejo de realizar. Para mayor información de esta interfaz puedes revisar el [https://docs.oracle.com/javase/7/docs/api/java/lang/Runnable.html||API de Java].



## La clase Thread

La clase Thread es una abstracción de todo lo que se puede hacer con un hilo de ejecución, por ejemplo: ejecutarlo, dormirlo, detenerlo, etc. Para mayor información de lo que ya tiene implementado la clase Thread, revisar el [https://docs.oracle.com/javase/7/docs/api/java/lang/Thread.html||API de Java].

De tal manera que basta con heredar (extends Thread) de la clase Thread para que la clase heredera tenga todas las cualidades de un hilo de ejecución.

 

Sin embargo es importante mencionar que la clase Thread, implementa la interfaz Runnable, es decir que una vez que hayamos heredado de la clase Thread es necesario redefinir el método run().

##¿Usar Runnable o Thread?

Sin importar cual de las 2 opciones se elija (Runnable ó Thread) vale la pena mencionar que para fines prácticos ambas funcionan igual, es decir que podemos poner en ejecución un hilo empleando el método run().

La principal diferencia radica en el hecho de que al heredar de la clase Thread, se tienen mas control sobre los hilos de ejecución y por el contrario al implementar la interfaz Runnable, unicamente se tiene un método directamente relacionado con los hilos de ejecución, el método run().

Otra diferencia es que al heredar de la clase Thread, ya no es posible heredar de otra clase. Por otro lado no importa si implementamos Runnable, Java permite implementar múltiples interfaces.

#Referencias

1. Yuri N. Skiba: Introducción a los métodos numéricos, Dirección General de Publicaciones U.N.A.M.
2. Ward Cheney, David Kincaid: Métodos Numéricos y Computación, Cenage Learning.
3. Dongarra Foster: SourceBook of parallel computing.
4. Riswan Butt: Numerical Analysys Using Matlab, Jones and Bartlett.
5. Universidad Granada: http://lsi.ugr.es/jmantas/pdp/tutoriales/tutorial_mpi.php.